### We're about to create and use our own MCP Server and MCP Client!

And we're right back in Cursor again, of course, and we're going into the Week 6 folder. And we're going to Lab 2 to create our own MCP server and client, which, as I quote here, it's pretty simple, but it's not super simple. The reason people are excited about MCP is that it's so simple to use other tools, not necessarily to create MCP servers. The first thing we're going to do, though, is we're going to look at a Python module. And I know that many people would much rather spend more time with Python modules than in labs. And this week, we'll be doing both. We'll be starting in labs. We will be moving to just Python code in modules. And there is a Python module called accounts.py. 

It's pretty simple, but it's not super-simple. The excitment around MCP is about how easy it is to share and use other MCP Servers - making our own does involve a bit of work.

Let's review some python code made mostly by a hard-working Engineering Team:

**Python module `accounts.py`**

And we're right back in Cursor again, of course, and we're going into the Week 6 folder. And we're going to Lab 2 to create our own MCP server and client, which, as I quote here, it's pretty simple, but it's not super simple. The reason people are excited about MCP is that it's so simple to use other tools, not necessarily to create MCP servers.

The first thing we're going to do, though, is we're going to look at a Python module. And I know that many people would much rather spend more time with Python modules than in labs. And this week, we'll be doing both. We'll be starting in labs. We will be moving to just Python code in modules.

And there is a Python module called accounts.py. And it contains a ton of code for managing your account, where you can buy and sell shares, and where you have a balance, and where you can do things like calculate profit and loss, list transactions, and where you can do things like buy shares and sell shares based on a market price. Looks like some interesting business logic.

```python
class Account(BaseModel):
    name: str
    balance: float
    strategy: str
    holdings: dict[str, int]
    transactions: list[Transaction]
    portfolio_value_time_series: list[tuple[str, float]]

    def buy_shares(self, symbol: str, quantity: int, rationale: str) -> str:
        ...
    def sell_shares(self, symbol: str, quantity: int, rationale: str) -> str:
        ...
    def calculate_portfolio_value(self):
        ...
    def list_transactions(self):
        ...
```

And hopefully, you remember, it seems familiar to you because this is, of course, code generated by our agent team in Week 3. Our engineering team in Crew AI created this code right here. And you can tell it's because of comments and type hints. And you probably know that I'm a bit hacky with this stuff, and that I only write comments and type hints when I'm forced. But here, our agent crew did a wonderful job. So it's cool that we're taking the code here that is being used by our agents.

Now, I made a slight change to it. I updated it so that it does save accounts in a database. And I've separated that out into a separate module called database.py. And this is just simply using SQLite. So this is very vanilla, but it allows you to read and write accounts as JSON objects. And that I've hooked up to accounts.py.

```python
@classmethod
def get(cls, name: str):
    fields = read_account(name.lower())
    if not fields:
        fields = {
            "name": name.lower(),
            "balance": INITIAL_BALANCE,
            "strategy": "",
            "holdings": {},
            "transactions": [],
            "portfolio_value_time_series": []
        }
        write_account(name, fields)
    return cls(**fields)

def save(self):
    write_account(self.name.lower(), self.model_dump())
```

But apart from making tiny changes like that, it is basically untouched code written by our engineering team of agents in Week 3. All right, so back to our lab for today. I see it says here Week 2, Day 2. I think we'll change that right away. That seems better.

Okay, so let's get started. We'll do some imports. We are now going to import this Python module account.

```python
from accounts import Account
```

Okay, so now I can call account.get. And if I pass in a name, it gets the account with that ID. Here I am. This is my account. Apparently, I have a balance of \$9,400. I have three Amazon shares, and I have some transactions. This is coming back from the code written by our agents.

```python
account = Account.get("Ed")
print(account)
```

And I can call buy shares. I can buy three more shares of Amazon, and I can give a reason. And we'll just rewind the clocks a bit and imagine this is a few years back. And I'll say I'm going to buy three shares of Amazon because this bookstore website looks promising. That's silly. So there we go. Yeah, if only I'd had that thought. Okay, so I bought three, and now I have six shares of Amazon in there.

```python
account.buy_shares("AMZN", 3, "Because this bookstore website looks promising")
```

And I can call account.report to get a report about it and all sorts of information about it. And I can also call account.listTransactions to see those transactions right there. So this is just showing that we've got some code. It was written by our agents, and we can operate it to do things like buying shares and listing transactions.

```python
print(account.report())
print(account.list_transactions())
```

Now it's MCP server time. So writing an MCP server is pretty easy. It's just boilerplate code that you use to wrap code you've already got into an MCP server, and you use some libraries provided by Anthropic. And let's have a look. I've created one, a Python module called accountserver.py. Let's see what happens. So it begins by importing an Anthropic class called FastMCP, and we also import our business logic, account. And we then create an MCP server by saying FastMCP accountserver. That is creating a new FastMCP server with that name.

```python
from fastmcp import FastMCP
from accounts import Account

mcp = FastMCP("accountserver")
```

What we then have is a number of functions listed here which are decorated with at MCP tool. And for each of them, we have a function like getBalance. We describe that function. We give information about it here using standard doc strings, and then we actually do that function. And in our case, we're just delegating to our business logic.

```python
@mcp.tool()
def getBalance(name: str) -> float:
    """Return the current balance for the account."""
    return Account.get(name).balance
```

But the idea is that this server will be spawned when we launch the MCP server. This will be launched, and these tools will all be tools that will be available, and they will work simply by calling the business logic we've imported right here. So we have getBalance, getHoldings, buyShares, sellShares, changeStrategy if we want to change the strategy associated with the portfolio.

```python
@mcp.tool()
def getHoldings(name: str):
    return Account.get(name).get_holdings()

@mcp.tool()
def buyShares(name: str, symbol: str, quantity: int, rationale: str):
    return Account.get(name).buy_shares(symbol, quantity, rationale)

@mcp.tool()
def sellShares(name: str, symbol: str, quantity: int, rationale: str):
    return Account.get(name).sell_shares(symbol, quantity, rationale)

@mcp.tool()
def changeStrategy(name: str, strategy: str):
    return Account.get(name).change_strategy(strategy)
```

And then I've also got some resources here. As I say, resources are not as common as tools, but I do want to show you how they work. And here we have the ability to access a resource. The name of an account will just return its report, and the strategy of an account will return its strategy. And it gets like a sort of a fake URL, a URL like this, where you can describe what resource you want to provide if something requests this resource.

```python
@mcp.resource("account://{name}/report")
def account_report(name: str):
    return Account.get(name).report()

@mcp.resource("account://{name}/strategy")
def account_strategy(name: str):
    return Account.get(name).get_strategy()
```

And then down at the very bottom, we have the final piece here, which is that when this script, this Python script is run, what it should actually do is call the run function on the MCP and say that my transport mechanism is the usual stdio. And because we've done that, when this Python script is run, it will launch that MCP server, it will import our business logic, and it will be ready to handle any of these tools.

```python
if __name__ == "__main__":
    mcp.run(transport="stdio")
```

And as you can see, there's not much to this at all. It's not super simple, but it's not hard at all to write your own MCP server that wraps your business logic and launches the server like that. And now we just have to try and put it to use.




In [2]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace
from agents.mcp import MCPServerStdio
from IPython.display import display, Markdown

load_dotenv(override=True)

True

Okay, so let's get started. We'll do some imports. We are now going to import this Python module account.

In [3]:
from accounts import Account

Okay, so now I can call account.get. And if I pass in a name, it gets the account with that ID. Here I am. This is my account. Apparently, I have a balance of \$9,400. I have three Amazon shares, and I have some transactions. This is coming back from the code written by our agents.

In [4]:
account = Account.get("Ed")
account

Account(name='ed', balance=9798.598, strategy='', holdings={'AMZN': 3}, transactions=[3 shares of AMZN at 67.134 each.], portfolio_value_time_series=[('2025-08-11 08:42:04', 9936.598), ('2025-08-11 08:42:27', 10050.598)])

And I can call buy shares. I can buy three more shares of Amazon, and I can give a reason. And we'll just rewind the clocks a bit and imagine this is a few years back. And I'll say I'm going to buy three shares of Amazon because this bookstore website looks promising. That's silly. So there we go. Yeah, if only I'd had that thought. Okay, so I bought three, and now I have six shares of Amazon in there.

In [5]:
account.buy_shares("AMZN", 3, "Because this bookstore website looks promising")

'Completed. Latest details:\n{"name": "ed", "balance": 9636.274, "strategy": "", "holdings": {"AMZN": 6}, "transactions": [{"symbol": "AMZN", "quantity": 3, "price": 67.134, "timestamp": "2025-08-11 08:42:04", "rationale": "Because this bookstore website looks promising"}, {"symbol": "AMZN", "quantity": 3, "price": 54.108, "timestamp": "2025-08-11 09:11:29", "rationale": "Because this bookstore website looks promising"}], "portfolio_value_time_series": [["2025-08-11 08:42:04", 9936.598], ["2025-08-11 08:42:27", 10050.598], ["2025-08-11 09:11:29", 9720.274]], "total_portfolio_value": 9720.274, "total_profit_loss": -279.72600000000057}'

And I can call account.report to get a report about it and all sorts of information about it.

In [6]:
account.report()

'{"name": "ed", "balance": 9636.274, "strategy": "", "holdings": {"AMZN": 6}, "transactions": [{"symbol": "AMZN", "quantity": 3, "price": 67.134, "timestamp": "2025-08-11 08:42:04", "rationale": "Because this bookstore website looks promising"}, {"symbol": "AMZN", "quantity": 3, "price": 54.108, "timestamp": "2025-08-11 09:11:29", "rationale": "Because this bookstore website looks promising"}], "portfolio_value_time_series": [["2025-08-11 08:42:04", 9936.598], ["2025-08-11 08:42:27", 10050.598], ["2025-08-11 09:11:29", 9720.274], ["2025-08-11 09:11:29", 9726.274]], "total_portfolio_value": 9726.274, "total_profit_loss": -273.72600000000057}'

And I can also call account.listTransactions to see those transactions right there. So this is just showing that we've got some code. It was written by our agents, and we can operate it to do things like buying shares and listing transactions.


In [7]:
account.list_transactions()

[{'symbol': 'AMZN',
  'quantity': 3,
  'price': 67.134,
  'timestamp': '2025-08-11 08:42:04',
  'rationale': 'Because this bookstore website looks promising'},
 {'symbol': 'AMZN',
  'quantity': 3,
  'price': 54.108,
  'timestamp': '2025-08-11 09:11:29',
  'rationale': 'Because this bookstore website looks promising'}]

#### `accounts_server.py`

Now it's MCP server time. So writing an MCP server is pretty easy. It's just boilerplate code that you use to wrap code you've already got into an MCP server, and you use some libraries provided by Anthropic. And let's have a look. I've created one, a Python module called accountserver.py. Let's see what happens. 


So it begins by importing an Anthropic class called FastMCP, and we also import our business logic, account. And we then create an MCP server by saying FastMCP accountserver. That is creating a new FastMCP server with that name.

```python
from fastmcp import FastMCP
from accounts import Account

mcp = FastMCP("accountserver")
```

What we then have is a number of functions listed here which are decorated with at MCP tool. And for each of them, we have a function like getBalance. We describe that function. We give information about it here using standard doc strings, and then we actually do that function. And in our case, we're just delegating to our business logic.

```python
@mcp.tool()
def getBalance(name: str) -> float:
    """Return the current balance for the account."""
    return Account.get(name).balance
```

But the idea is that this server will be spawned when we launch the MCP server. This will be launched, and these tools will all be tools that will be available, and they will work simply by calling the business logic we've imported right here. So we have getBalance, getHoldings, buyShares, sellShares, changeStrategy if we want to change the strategy associated with the portfolio.

```python
@mcp.tool()
def getHoldings(name: str):
    return Account.get(name).get_holdings()

@mcp.tool()
def buyShares(name: str, symbol: str, quantity: int, rationale: str):
    return Account.get(name).buy_shares(symbol, quantity, rationale)

@mcp.tool()
def sellShares(name: str, symbol: str, quantity: int, rationale: str):
    return Account.get(name).sell_shares(symbol, quantity, rationale)

@mcp.tool()
def changeStrategy(name: str, strategy: str):
    return Account.get(name).change_strategy(strategy)
```

And then I've also got some resources here. As I say, resources are not as common as tools, but I do want to show you how they work. And here we have the ability to access a resource. The name of an account will just return its report, and the strategy of an account will return its strategy. And it gets like a sort of a fake URL, a URL like this, where you can describe what resource you want to provide if something requests this resource.

```python
@mcp.resource("account://{name}/report")
def account_report(name: str):
    return Account.get(name).report()

@mcp.resource("account://{name}/strategy")
def account_strategy(name: str):
    return Account.get(name).get_strategy()
```

And then down at the very bottom, we have the final piece here, which is that when this script, this Python script is run, what it should actually do is call the run function on the MCP and say that my transport mechanism is the usual stdio. And because we've done that, when this Python script is run, it will launch that MCP server, it will import our business logic, and it will be ready to handle any of these tools.

```python
if __name__ == "__main__":
    mcp.run(transport="stdio")
```

And as you can see, there's not much to this at all. It's not super simple, but it's not hard at all to write your own MCP server that wraps your business logic and launches the server like that. And now we just have to try and put it to use.

### Now we write an MCP server and use it directly!

So here we are back in the lab. And what we're looking at here are parameters in our very own MCP server that we just wrote. So the command is uvrun, because that's exactly what we would type at a command line to run this module. uvrun accountserver.py Just run the accountserver.py that I just showed you. And we know that when that's called, it's going to create an MCP server, a fast MCP. It's going to call run, and then we've got some functions decorated as MCP tools. All right? So then we use the OpenAI Agents SDK Context Manager with MCP Server Studio. And we pass in those parameters. Remember this timeout. And we then are going to call server.listtools.

In [8]:
# Now let's use our accounts server as an MCP server

params = {"command": "uv", "args": ["run", "accounts_server.py"]}
async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as server:
    mcp_tools = await server.list_tools()


So just by running this, what's going to happen is it's going to create an MCP client. It's then going to spawn our MCP server by carrying out this instruction right here. And it's then going to ask it, what tools do you offer us? And we'll print them out. And we're hoping to see the things that we decorated. Let's see if this works. Can it be that simple? Off it goes. Took 1.4 seconds. Let's just print that. And there we go. These are the functions that we just decorated. Get balance, get holdings, buy shares, sell shares, change strategy. It's available. Wow.

In [9]:
mcp_tools

[Tool(name='get_balance', description='Get the cash balance of the given account name.\n\n    Args:\n        name: The name of the account holder\n    ', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_balanceArguments', 'type': 'object'}, annotations=None),
 Tool(name='get_holdings', description='Get the holdings of the given account name.\n\n    Args:\n        name: The name of the account holder\n    ', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_holdingsArguments', 'type': 'object'}, annotations=None),
 Tool(name='buy_shares', description="Buy shares of a stock.\n\n    Args:\n        name: The name of the account holder\n        symbol: The symbol of the stock\n        quantity: The quantity of shares to buy\n        rationale: The rationale for the purchase and fit with the account's strategy\n    ", inputSchema={'properties': {'name': {'title': 'Name', '

So as you can see, it's not super easy, but it's also pretty easy. Okay. Let's try and put this to action. Let's have some instructions. You're able to manage an account for a client and answer questions about the account. My name's Ed. My account is under the name Ed. What is my balance and my holdings? So it's going to need to make use of these tools. And we'll give it the latest model. Why not? Let's take that. And now this, again, is the same code as before. We use the context manager with MCP Server Studio. We pass in our parameters. Let's put in that timeout. Client session timeout is 30. And we will pass in our instructions, our model. And we will then runner.run and display the output. We're hoping that it's going to spawn the server, call the tools, and be able to tell me about my six Amazon shares and the balance and everything like that. So, Ed, your current cash balance is 8,000. Your holdings include six shares of Amazon. If you need any further details, then let me know. So that is us successfully calling our very own MCP server that then calls our business logic.

In [10]:
instructions = "You are able to manage an account for a client, and answer questions about the account."
request = "My name is Ed and my account is under the name Ed. What's my balance and my holdings?"
model = "gpt-4.1-mini"

In [11]:

async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as mcp_server:
    agent = Agent(name="account_manager", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("account_manager"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))


Ed, your current account balance is $9,636.27. Your holdings include 6 shares of Amazon (AMZN). Is there anything specific you would like to do next?

### Now let's build our own MCP Client

And now it's time to show you what it's like to write an MCP client. But I should first explain that it's not a common task to write an MCP client. You don't really need to do it anymore. When I first started working on this lab, the way that you work with OpenAI Agents SDK, they didn't sort of natively support MCP. You had to write your own client and then provide the tools into OpenAI Agents SDK. And it was the day that I finished this project and checked it into GitHub and did the push, that same day, about two or three hours later, they released an update to OpenAI Agents SDK that simplified everything. All my code was useless. You can look at the Git timestamps if you don't believe me. It was crazy. I was infuriated. But it is a great thing and basically means that just with this context manager construct, you automatically get OpenAI Agents SDK creating the client for you.

```py
async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as mcp_server:
    agent = Agent(name="account_manager", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("account_manager"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))
```

 But anyway, it's a good exercise to show you what it's like to make a client. And also, this construct works for tools, but if you want to use resources, I think you still need to write a client. It's not so common, but we'll do it anyway and you'll see how it works. 


### `accounts_client.py.`

Okay, so this is in a Python module called accounts-client.py. And this is where the magic happens that is no longer needed, really. So this is an MCP client for use with my accounts MCP server.

```python
import mcp
from mcp.client.stdio import stdio_client
from mcp import StdioServerParameters
from agents import FunctionTool
import json

params = StdioServerParameters(command="uv", args=["run", "accounts_server.py"], env=None)
```

So at the top here, I specified these are the parameters that will be used to launch the MCP server. Now, this could be something configurable. You can make a sort of generic MCP client that takes this as configuration and then spawns it. Now, this is just a fixed client for our accounts MCP server. And so this is an accounts MCP client. And so there are a few things we need to be able to do. We need to be able to list the tools.

```python
async def list_accounts_tools():
    async with stdio_client(params) as streams:
        async with mcp.ClientSession(*streams) as session:
            await session.initialize()
            tools_result = await session.list_tools()
            return tools_result.tools
```

And so this first function here is an example of the function that lists tools. And we're basically using a bunch of anthropic code here for our client. And you can see that there's some context managers that we manage a session. We initialize the session, and then we call list tools on the session and return the tools. And so it's just sort of plumbing stuff that you need to know about if you want to write something that will contact your server and list the tools.

```python
async def call_accounts_tool(tool_name, tool_args):
    async with stdio_client(params) as streams:
        async with mcp.ClientSession(*streams) as session:
            await session.initialize()
            result = await session.call_tool(tool_name, tool_args)
            return result
```

This second one is actually calling a tool. This is how you go about reading one of the resources.

```python
async def read_accounts_resource(name):
    async with stdio_client(params) as streams:
        async with mcp.ClientSession(*streams) as session:
            await session.initialize()
            result = await session.read_resource(f"accounts://accounts_server/{name}")
            return result.contents[0].text
```

And this is reading the other resources.

```python
async def read_strategy_resource(name):
    async with stdio_client(params) as streams:
        async with mcp.ClientSession(*streams) as session:
            await session.initialize()
            result = await session.read_resource(f"accounts://strategy/{name}")
            return result.contents[0].text
```

You can see I've specified the resource name right there. And then finally, the types, the way that MCP returns tools is very similar to the JSON, the standard JSON that you use when you call a tool as we did in week one. But it's not identical. There is some couple of slight differences. So if you're writing your own MCP client, you also have to know how to map between the MCP description of a tool and the kind of JSON description of a tool that's used generally when you're calling LLMs.

```python
async def get_accounts_tools_openai():
    openai_tools = []
    for tool in await list_accounts_tools():
        schema = {**tool.inputSchema, "additionalProperties": False}
        openai_tool = FunctionTool(
            name=tool.name,
            description=tool.description,
            params_json_schema=schema,
            on_invoke_tool=lambda ctx, args, toolname=tool.name: call_accounts_tool(toolname, json.loads(args))
        )
        openai_tools.append(openai_tool)
    return openai_tools
```

And that is what this function here does that I painstakingly wrote. But all of this comes for free packaged in OpenAI HSSDK, so you don't need to do all of this. But you can look through it should you be interested and should you want to understand what does it mean to have your own MCP client.






---

So I import from that module we just looked at some of the functions we just saw. And the first function I call is the list account tools function, which is the one that's going to spawn an MCP server and ask it for its tools. That will come back as MCP tools. And then I'm going to call the other function I showed you where I take the MCP tools and I reconstitute them as OpenAI tools. I turn them into these function tools just as if I had a function and decorated it with app function tools. Same thing. And then I'm going to print that. 

In [12]:
from accounts_client import get_accounts_tools_openai, read_accounts_resource, list_accounts_tools

mcp_tools = await list_accounts_tools()
print(mcp_tools)
openai_tools = await get_accounts_tools_openai()
print(openai_tools)

[Tool(name='get_balance', description='Get the cash balance of the given account name.\n\n    Args:\n        name: The name of the account holder\n    ', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_balanceArguments', 'type': 'object'}, annotations=None), Tool(name='get_holdings', description='Get the holdings of the given account name.\n\n    Args:\n        name: The name of the account holder\n    ', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_holdingsArguments', 'type': 'object'}, annotations=None), Tool(name='buy_shares', description="Buy shares of a stock.\n\n    Args:\n        name: The name of the account holder\n        symbol: The symbol of the stock\n        quantity: The quantity of shares to buy\n        rationale: The rationale for the purchase and fit with the account's strategy\n    ", inputSchema={'properties': {'name': {'title': 'Name', 'ty

So I'll print these two separately so you can see the results side by side. So this first line here, this is the actual MCP tools themselves with their description, the arguments of all the tools. And you can take a look at them should you wish. And underneath it is the same thing that reconstituted as a function tool, which is the kind of object that needs to be passed in to OpenAI.

`mcp_servers=[mcp_server`

```py
async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as mcp_server:
    agent = Agent(name="account_manager", instructions=instructions, model=model, mcp_servers=[mcp_server])
```
And indeed, that's what I'm going to do right here. So this is the way you had to do it before OpenAI built this so that now you can just pass in the MCP servers. You don't have to build a client and do all of this stuff. But before that, in fact, very recently, this was the way to do it. You'd actually have to pass in the tools themselves. So we used our MCP clients to find the tools, and then we're passing them in here. So as I say, you just follow along for interest to see what this is doing, but it's not like you need to do this yourself.

`tools=openai_tools`

```py
with trace("account_mcp_client"):
    agent = Agent(name="account_manager", instructions=instructions, model=model, tools=openai_tools)
```

In [13]:
request = "My name is Ed and my account is under the name Ed. What's my balance?"

with trace("account_mcp_client"):
    agent = Agent(name="account_manager", instructions=instructions, model=model, tools=openai_tools)
    result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

Hi Ed, your current cash balance is $9,636.27. If you need any more information or assistance with your account, feel free to ask!

So as always, I've got a trace, and then I'll pass in my instructions. And I will pass in the tools that when these tools are called, it's actually going to call the MCP client. It's going to call the MCP server. It's going to actually run our business logic. Okay, so now we're going to run our MCP clients like this. So we're passing in the tools. We are calling the model, and it has now responded. I asked, what's my balance? And it's responded with a balance. And so just as a recap of what's going on there, we provided it with some tools right here. Those tools are, in fact, wrappers around MCP tools, which actually created an MCP client, which launched our MCP server, and which ran our function through the business logic on the MCP server and got back the results to the LLM. So quite a lot was going on there behind the scenes with this. As I say, you really shouldn't need to do this yourself because OpenAI Agents SDK does it all for you, but it's useful to see how the plumbing works behind the scenes. 

---

Now, here's another example of using the MCP client. This time, we're using this read accounts resource client function that I wrote. And I think that you do need to do it this way. If you want to use resources, then that doesn't come for free with the OpenAI Agents SDK package. So this is an example of reading the resource called Ed. So let's see what happens if we run that. Let's run it through our client, through our server, and that's what comes back, this description of my account. 

In [14]:
context = await read_accounts_resource("ed")
print(context)

{"name": "ed", "balance": 9636.274, "strategy": "", "holdings": {"AMZN": 6}, "transactions": [{"symbol": "AMZN", "quantity": 3, "price": 67.134, "timestamp": "2025-08-11 08:42:04", "rationale": "Because this bookstore website looks promising"}, {"symbol": "AMZN", "quantity": 3, "price": 54.108, "timestamp": "2025-08-11 09:11:29", "rationale": "Because this bookstore website looks promising"}], "portfolio_value_time_series": [["2025-08-11 08:42:04", 9936.598], ["2025-08-11 08:42:27", 10050.598], ["2025-08-11 09:11:29", 9720.274], ["2025-08-11 09:11:29", 9726.274], ["2025-08-11 09:11:43", 9846.274]], "total_portfolio_value": 9846.274, "total_profit_loss": -153.72600000000057}


And just to show you, I could also, of course, just import the business logic directly, and this is ending up just calling this function report. So this should give exactly the same answer. You can see the same stuff. So essentially, what we've done is we've taken this piece of business logic. We've wrapped it in an MCP server to be available as a certain resource. We've written an MCP client that allows you to expose that resource, and that is what this function is. And so you can call this MCP client to get this. You could also just call the business logic directly. And so why would you want to do this? Again, it's if you wanted to share this resource with other people. It gives everyone a simple, streamlined way to access your resources rather than having to understand how your business logic works. Okay.

In [15]:
from accounts import Account
Account.get("ed").report()

'{"name": "ed", "balance": 9636.274, "strategy": "", "holdings": {"AMZN": 6}, "transactions": [{"symbol": "AMZN", "quantity": 3, "price": 67.134, "timestamp": "2025-08-11 08:42:04", "rationale": "Because this bookstore website looks promising"}, {"symbol": "AMZN", "quantity": 3, "price": 54.108, "timestamp": "2025-08-11 09:11:29", "rationale": "Because this bookstore website looks promising"}], "portfolio_value_time_series": [["2025-08-11 08:42:04", 9936.598], ["2025-08-11 08:42:27", 10050.598], ["2025-08-11 09:11:29", 9720.274], ["2025-08-11 09:11:29", 9726.274], ["2025-08-11 09:11:43", 9846.274], ["2025-08-11 09:11:43", 9726.274]], "total_portfolio_value": 9726.274, "total_profit_loss": -273.72600000000057}'

<table style="margin: 0; text-align: left; width:100%">
        <td>
            <h2 style="color:#ff7800;">Exercises</h2>
            <span style="color:#ff7800;">Make your own MCP Server! Make a simple function to return the current Date, and expose it as a tool so that an Agent can tell you today's date.<br/>Harder optional exercise: then make an MCP Client, and use a native OpenAI call (without the Agents SDK) to use your tool via your client.
            </span>
        </td>
    </tr>
</table>


Okay, that wraps up creating MCP servers and MCP clients, with most of the emphasis on the servers. And I would now like to give you an exercise to go and create your own. So one super simple one to do would be to write an MCP server that can tell you the current date, give the current date, and you can expose it as a tool that you can equip OpenAI agent with so that it can find out the current date and it can make sure that what is the content of the question it's answering, it's doing so mindful of what the current date is. And, yeah, then as a part of that exercise, you could not only build that MCP server, but also build an MCP client to accompany it, taking a look at exactly how I just did that for the accounts client, compared with the accounts server, same approach. And then you could actually write something that makes like a native call to OpenAI, as we did in week one, like a simple native call with a tool as a JSON, and then you're getting really bare metal, and you're seeing how you can call an NLM and have an MCP client and an MCP server where you're writing all the nuts and bolts. The only real use of that is to give you some direct exposure to it, so only do that if you're interested. You won't have to do that day to day, but it's an interesting exercise. It's also worth pointing out that actually answering the current date isn't a great tool, isn't super useful in the real world, because if you need to tell your agent about the current date, it's better just to state it in the prompt so that it always has it available, so that the model doesn't have to go through the extra work of knowing to call your tool to collect the date. So, if you feel like it, try to make some more interesting tools. Have a shot at making a calculator that can do some calculation operation on two inputs, or something like that. So try and make some interesting tool that appeals to you, and then write an MCP server, and perhaps also explore the MCP client, should you wish. Enjoy that, and we will then go over to wrap up for today.